## Разведочный анализ данных (exploratory data analysis, EDA)

Задачи, которые ставятся перед аналитиками, достаточно разнообразны. Однако всё начинается с данных.

Нужно понимать, что данные не берутся "из воздуха". Как и задачи, связанные с ними. В книге Билла Фрэнкса об операционной аналитике автор акцентирует внимание на том, что непродуманные инвестиции в сбор и хранение данных по принципу "а вдруг потом пригодятся" зачастую себя не оправдывают. Только после того, как поставлена определённая цель, можно начинать процесс сбора (или, возможно, покупки) и анализа данных.

К сожалению, на практике данные в "сыром" виде обычно малопригодны для анализа. Процесс подготовки и очистки данных (препроцессинг, англ. data preparation, pre-processing, data cleaning) может быть весьма трудоёмким и по времени занимать больше, чем собственно построение и валидация моделей на основе данных. Выделим некоторые составляющие этого процесса:

- data specification (понимание данных)
- data editing (редактирование данных, исправление ошибок --- ручное, автоматическое или их комбинация)
- работа с пропущенными значениями
- нормализация
- feature extraction and selection (создание и отбор признаков)

В результате получаем данные в удобном для анализа формате, как правило, табличном. Таблица (или датафрейм) имеет структуру "объекты-признаки": строки соответствуют отдельным сущностям (объектам, примерам, экземплярам), а столбцы --- атрибутам этих сущностей (признакам).

## Подготовка

### Импорты

In [ ]:
import pandas as pd
import numpy as np

In [15]:
from datetime import datetime

In [ ]:
from sqlalchemy import create_engine
import psycopg2
#для связи с postgresql

In [16]:
import matplotlib.pyplot as plt

In [17]:
from pandas.plotting import register_matplotlib_converters

# конвертеры, которые позволяют использовать типы pandas в matplotlib 

In [ ]:
import seaborn as sns

In [ ]:
import plotly.express as px

In [ ]:
from plotly import graph_objects as go

In [ ]:
import math as mth

In [ ]:
import scipy.stats

In [ ]:
from scipy import stats as st

In [ ]:
import sys

In [ ]:
import getopt

In [ ]:
import csv
from io import StringIO

In [ ]:
import sqlite3

In [ ]:
import os

In [ ]:
import re

### Опции

In [ ]:
# pd.set_option('max_rows', 5)

# количество строк в таблице не больше 5, чтобы не писать head() за каждым фреймом
# при необходимости раскрыть из комментария полный вывод данных в колонке
# pd.set_option('display.max_colwidth', None)

In [ ]:
pd.set_option('display.float_format', '{:,.2f}'.format)
# разделитель ',' и два знака после запятой у чисел с плавающей точкой

#### опции для графики

In [ ]:
large = 16; med = 12; small = 10
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (12, 8),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}

plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')

In [ ]:
sns.set_palette('deep') 
sns.set_style("whitegrid")

### Заливка данных

In [ ]:
df = pd.read_csv('path_to_file')

In [ ]:
df = pd.read_csv('path_to_file', encoding='windows-1251', sep=';')

In [ ]:
df = pd.read_csv('path_to_file', encoding='utf8', sep=';')

In [1]:
# см. template_data_filling.ipynb
# см. template_data_load.ipynb

## EDA (3 этапа)

### 1. Первичный анализ датасета

- Знакомимся с таблицей данных
- Оцениваем ее объем
- Оцениваем признаки объектов, хранящихся в ней
- Выявляем наличие пропусков

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum()
# поиск пропущенных значений

In [ ]:
df.duplicated().sum()
#поиск абсолютных дубликатов

#### описательная статистика

In [ ]:
# Описательная статистика для числовых столбцов датасета
df.describe()

In [ ]:
# Описательная статистика для нечисловых столбцов датасета
df.describe(include=['object', 'bool'])

### 2. Анализ и визуализация отдельных колонок

Пытаемся сформулировать вопросы для колонок и ответить на них, используя
- описательная статистика
- выбросы, аномалии
- сортировка
- визуализация

#### уникальные значения

In [ ]:
df['name_column'].unique()
len(df['name_column'].unique())

#### срез фрейма по отдельным столбцам

In [ ]:
df[['name_column_1', 'name_column_2']]

In [ ]:
# метод loc, который ищет значения по их названию
df.loc[:, 'name_column_3': 'name_column_5']
# остались столбцы от 'name_column_3' до 'name_column_5'

In [ ]:
# метод iloc, который делает срез и по столбцам и по строкам
df.iloc[0:100, 3:5]
# первый параметр показывает индексы строк, которые останутся, второй — индексы столбцов.

#### описательная статистика по отдельным столбцам

In [ ]:
df['name_column'].describe()

In [ ]:
df['name_column'].value_counts()

In [ ]:
# Максимум, минимум, среднее

(df['name_column'].max(),
df['name_column'].min(),
df['name_column'].mean())

In [ ]:
agg_func_math = {
    'name_column_with_data': ['count', 'sum', 'mean', 'median', 'min', 'max', 'std', 'var', 'mad', 'prod']
}
# задаем список функций по определенному столбцу с расчетными данными

df.groupby('name_column', dropna=False).agg(agg_func_math).round(2)
# учитываем строки с NaN: dropna=False

In [ ]:
# describe вызывается описательная статистика
df.groupby('name_column')['name_column_with_data'].describe().round(2)

#### сортировка по столбцу

In [ ]:
df.sort_values('name_column')

#### 📈 визуализация

##### графический анализ ящик с усами - boxplot

In [ ]:
# import matplotlib.pyplot as plt
plt.boxplot(df['name_column'])
plt.show

In [ ]:
# import seaborn as sns

sns.boxplot(x='name_column_1', y='name_column_2', data=df)
plt.axis(ymin=0, ymax=100000)
# name_column_1 - категориальные или сегментированные непрерывные данные
# name_column_2 - непрерывные или сегментированные непрерывные данные
# ymin, ymax - ограничение по шкале y

##### графический анализ - Гистограмма

In [ ]:
# import matplotlib.pyplot as plt
plt.hist(df['name_column'].dropna(), bins=10, linewidth=5, edgecolor="white")
plt.xlim((None,55)) # для "отрезания" от графика неинформативного выброса справа (>55)
plt.show()
# dropna() - убираем отсутствующие значения,
# bins=10 - толщина столбца, чем больше цифра - тем тоньше

##### графический анализ - круговая диаграмма

In [ ]:
plt.pie(
    df['name_column'].value_counts().values,
    labels=df['name_column'].value_counts().index,
    wedgeprops=dict(width=0.5) # для бублика
    )
plt.title('<title>')
plt.show

In [ ]:
# круговая диаграмма с легендой и значениями
labels = [f"{n} ({v/df['name_column'].value_counts().values.sum():.1%})" for n,v in zip(df['name_column'].value_counts().index, df['name_column'].value_counts().values)]
# labels = [f"{n} {v:.1} руб." for n,v in zip(df['name_column'].value_counts().index, df['name_column'].value_counts().values)]
plt.pie(
    df['name_column'].value_counts().values,
    radius=1.1,
    explode=[0.15] + [0 for _ in range(len(df['name_column'].value_counts().index) - 1)]
    # посл.строка для разделения круговой диаграммы
    )
plt.legend(
    bbox_to_anchor = (-0.16, 0.45, 0.25, 0.25),
    loc = 'best', labels = labels )
plt.title('<title>')
plt.show

##### графический анализ - столбцовая диаграмма

In [ ]:
# bar - для вертикальных столбцов, barh - для горизонтальных
plt.bar(
    df['name_column'].value_counts().index,
    df['name_column'].value_counts().values
)
plt.title('<title>')
plt.show

### Предобработка

In [ ]:
# список столбцов в виде list()
df.columns.tolist()

#### переименование столбцов

In [ ]:
df = df.rename(columns=lambda x: x.lower().replace(' ', '_'))

#### удаление столбца

In [ ]:
df = df.drop('name_column', 1)

#### удаление дубликатов

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)
# удаление дубликатов

#### преобразование строк в дату и время

In [ ]:
df['name_column'] = pd.to_datetime(df['name_column'])

#### преобразование строк в число

In [ ]:
# import re
def clear_string(str):
    return int(re.sub('\D', '', str))

In [ ]:
# обработка датасета к.л. функцией
df['name_column'] = df['name_column'].apply(clear_string)

#### Категориальные переменные

##### бинарная переменная

In [ ]:
# Создаем бинарную переменную в зависимости от значения другого столбца

df['name_column'] = 0
for i in range(len(df.other_column)):
    if df.other_column[i] < 0: # или другое условие
        df['name_column'][i] = 1

##### диапазоны

In [ ]:
df['levels']=df['name_column'].apply(
    lambda x: 'low' if x<5.8 else 'middle' if x<6.6 else 'high'
    )

In [ ]:
df['levels'] = pd.cut(df['name_column'],bins=[3,5.8,6.6,9],right=False)
# cut  —  другой способ. Как и в предыдущем случае, делим значения на 3 группы

In [1]:
from scipy import stats
chi2, p_value, dof, ev = stats.chi2_contingency(([20,15],[11,12],[7,9]))
print(f'''
Хи квадрат {chi2}
p - value {p_value}
Степеней свобод {dof}
Ожидаемые наблюдения
{ev}
''')


Хи квадрат 0.9544070774762996
p - value 0.6205162173513055
Степеней свобод 2
Ожидаемые наблюдения
[[17.97297297 17.02702703]
 [11.81081081 11.18918919]
 [ 8.21621622  7.78378378]]

